# Handling Missing Values

Hapus nilai yang hilang, atau isi dengan alur kerja otomatis.

Pembersihan data adalah bagian penting dari ilmu data, tetapi bisa sangat membuat frustasi. Mengapa beberapa bidang teks Anda kacau? Apa yang harus Anda lakukan dengan nilai yang hilang tersebut? Mengapa tanggal Anda tidak diformat dengan benar? Bagaimana cara membersihkan entri data yang tidak konsisten dengan cepat? Dalam kursus ini, kamu akan belajar mengapa kamu mengalami masalah-masalah tersebut dan yang lebih penting lagi, bagaimana cara memperbaikinya!

Dalam kursus ini, kamu akan belajar cara mengatasi beberapa masalah pembersihan data yang paling umum sehingga kamu bisa menganalisis data dengan lebih cepat. Kamu akan mengerjakan lima latihan langsung dengan data yang berantakan dan menjawab beberapa pertanyaan pembersihan data yang paling sering ditanyakan.

Dalam buku catatan ini, kita akan melihat cara menangani nilai yang hilang.

## Pertama, Lihatlah Data yang Ada

Hal pertama yang harus kita lakukan adalah memuat pustaka dan dataset yang akan kita gunakan.

Untuk demonstrasi, kita akan menggunakan dataset peristiwa yang terjadi dalam pertandingan American Football. Pada latihan berikut, Anda akan menerapkan keterampilan baru Anda pada dataset izin bangunan yang dikeluarkan di San Francisco.

In [6]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")

# mengatur benih untuk reproduktifitas
np.random.seed(0) 

C:\Users\faizend\AppData\Local\Temp\ipykernel_17280\3706658807.py:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")


Hal pertama yang harus dilakukan ketika Anda mendapatkan dataset baru adalah melihat beberapa di antaranya. Hal ini memungkinkan Anda untuk melihat apakah semuanya terbaca dengan benar dan memberikan gambaran tentang apa yang terjadi pada data tersebut. Dalam hal ini, mari kita lihat apakah ada nilai yang hilang, yang akan direpresentasikan dengan NaN atau None.

In [26]:
# Lihatlah lima baris pertama dari file nfl_data.
# Saya dapat melihat beberapa data yang hilang!
nfl_data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Ya, sepertinya ada beberapa nilai yang hilang.

## Berapa banyak titik data yang hilang yang kita miliki?

Baiklah, sekarang kita tahu bahwa kita memiliki beberapa nilai yang hilang. Mari kita lihat berapa banyak yang kita miliki di setiap kolom.

In [8]:
# dapatkan jumlah titik data yang hilang per kolom
missing_values_count = nfl_data.isnull().sum()

# lihat jumlah titik yang hilang dalam sepuluh kolom pertama
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Sepertinya banyak sekali! Mungkin akan sangat membantu jika kita melihat berapa persen nilai dalam dataset kita yang hilang untuk memberikan gambaran yang lebih baik tentang skala masalah ini:

In [33]:
# berapa banyak total nilai yang hilang yang kita miliki?
total_cells = np.prod(nfl_data.shape)
total_missing = missing_values_count.sum()

# persen dari data yang hilang
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

27.66722370547874


Wow, hampir seperempat sel dalam dataset ini kosong! Pada langkah berikutnya, kita akan melihat lebih dekat pada beberapa kolom dengan nilai yang hilang dan mencoba mencari tahu apa yang mungkin terjadi dengannya.

## Cari tahu mengapa data hilang

Ini adalah titik di mana kita masuk ke bagian ilmu data yang saya suka sebut sebagai "data intution", yang saya maksud adalah "benar-benar melihat data Anda dan mencoba mencari tahu mengapa data tersebut seperti itu dan bagaimana hal itu akan mempengaruhi analisis Anda". Ini bisa menjadi bagian yang membuat frustasi dalam ilmu data, terutama jika Anda masih baru di bidang ini dan tidak memiliki banyak pengalaman. Untuk menangani nilai yang hilang, Anda harus menggunakan intuisi Anda untuk mencari tahu mengapa nilai tersebut hilang. Salah satu pertanyaan terpenting yang dapat Anda tanyakan pada diri sendiri untuk membantu mencari tahu hal ini adalah ini:

> Apakah nilai ini hilang karena tidak dicatat atau karena tidak ada?

Jika sebuah nilai hilang karena tidak ada (seperti tinggi anak tertua dari seseorang yang tidak memiliki anak), maka tidak masuk akal untuk mencoba menebak-nebak nilai tersebut. Nilai-nilai ini mungkin ingin Anda simpan sebagai `NaN`. Di sisi lain, jika sebuah nilai hilang karena tidak dicatat, maka Anda dapat mencoba menebak apa yang mungkin terjadi berdasarkan nilai-nilai lain dalam kolom dan baris tersebut. Ini disebut **imputasi**, dan kita akan belajar bagaimana melakukannya selanjutnya! :)

Mari kita kerjakan sebuah contoh. Melihat jumlah nilai yang hilang dalam dataframe `nfl_data`, saya melihat bahwa kolom "TimesSec" memiliki banyak nilai yang hilang di dalamnya:

In [34]:
# lihat jumlah titik yang hilang dalam sepuluh kolom pertama
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Dengan melihat dokumentasi, saya dapat melihat bahwa kolom ini memiliki informasi tentang jumlah detik yang tersisa dalam permainan ketika permainan dibuat. Ini berarti bahwa nilai-nilai ini mungkin hilang karena tidak dicatat, bukan karena tidak ada. Jadi, masuk akal jika kita mencoba menebak berapa nilai yang seharusnya daripada membiarkannya sebagai NA.

Di sisi lain, ada field lain, seperti "PenalizedTeam" yang juga memiliki banyak field yang hilang. Dalam kasus ini, bagaimanapun, bidang tersebut hilang karena jika tidak ada penalti maka tidak masuk akal untuk mengatakan tim mana yang dihukum. Untuk kolom ini, akan lebih masuk akal untuk membiarkannya kosong atau menambahkan nilai ketiga seperti "tidak ada" dan menggunakannya untuk menggantikan NA.

> Saran: Ini adalah tempat yang tepat untuk membaca dokumentasi dataset jika Anda belum membacanya! Jika Anda bekerja dengan dataset yang Anda dapatkan dari orang lain, Anda juga dapat mencoba menghubungi mereka untuk mendapatkan informasi lebih lanjut.

Jika Anda melakukan analisis data dengan sangat hati-hati, ini adalah titik di mana Anda akan melihat setiap kolom satu per satu untuk mengetahui strategi terbaik untuk mengisi nilai yang hilang. Untuk sisa dari buku catatan ini, kita akan membahas beberapa teknik "cepat dan kotor" yang dapat membantu Anda dengan nilai yang hilang, tetapi mungkin juga akan menghilangkan beberapa informasi yang berguna atau menambahkan beberapa noise ke data Anda.

## Menghilangkan nilai yang hilang

Jika Anda sedang terburu-buru atau tidak memiliki alasan untuk mencari tahu mengapa nilai Anda hilang, salah satu pilihan yang Anda miliki adalah menghapus baris atau kolom yang berisi nilai yang hilang. (Catatan: Saya biasanya tidak merekomendasikan cara ini untuk proyek-proyek penting! Biasanya sangat berharga untuk meluangkan waktu untuk memeriksa data Anda dan benar-benar melihat semua kolom dengan nilai yang hilang satu per satu untuk benar-benar mengenal kumpulan data Anda).

Jika Anda yakin ingin menghapus baris dengan nilai yang hilang, pandas memiliki fungsi yang berguna, dropna() untuk membantu Anda melakukan ini. Mari kita coba pada dataset NFL kita!

In [35]:
# hapus semua baris yang berisi nilai yang hilang
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Oh sayang, sepertinya hal itu telah menghapus semua data kita! 😱 Ini karena setiap baris dalam kumpulan data kita memiliki setidaknya satu nilai yang hilang. Kita mungkin akan lebih baik jika menghapus semua kolom yang memiliki setidaknya satu nilai yang hilang.

In [36]:
# hapus semua kolom dengan setidaknya satu nilai yang hilang
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009


In [37]:
# Berapa banyak data yang hilang?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 37


Kami telah kehilangan cukup banyak data, tetapi pada titik ini kami telah berhasil menghapus semua NaN dari data kami.

## Mengisi nilai yang hilang secara otomatis

Pilihan lainnya adalah mencoba mengisi nilai yang hilang. Untuk bagian berikutnya, saya mendapatkan sub-bagian kecil dari data NFL sehingga dapat dicetak dengan baik.

In [38]:
# dapatkan sebagian kecil dari dataset NFL
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Kita dapat menggunakan fungsi `fillna()` dari Panda untuk mengisi nilai yang hilang dalam sebuah dataframe untuk kita. Salah satu opsi yang kita miliki adalah menentukan dengan apa kita ingin nilai `NaN` diganti. Di sini, saya mengatakan bahwa saya ingin mengganti semua nilai `NaN` dengan 0.

In [39]:
# ganti semua NA dengan 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


Saya juga bisa sedikit lebih cerdas dan mengganti nilai yang hilang dengan nilai apa pun yang muncul langsung setelahnya di kolom yang sama. (Hal ini sangat masuk akal untuk kumpulan data yang pengamatannya memiliki urutan logis).

In [46]:
# ganti semua nilai NA yang berada tepat setelahnya di kolom yang sama,
# lalu ganti semua na yang tersisa dengan 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

C:\Users\faizend\AppData\Local\Temp\ipykernel_17280\321882698.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)


,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009
